In [145]:
import pandas as pd

# 1. Load data

In [306]:
data = pd.read_csv('/home/labhhc/Documents/Project/CADD_package/Data_preprocess/Data/CHEMBL392.csv', sep=';')
data.head(5)

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Name,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties
0,CHEMBL4213237,NaN,None,286.46,0,2.19,7'e,CCCCCCCCCCCNC[C@@H]1OC[C@H](N)[C@H]1O,IC50,'=',...,A549,Homo sapiens,CELL-LINE,CHEMBL4184090,1,Scientific Literature,Bioorg Med Chem Lett,2018.0,CHEMBL3307651,NaN
1,CHEMBL4204703,NaN,None,308.39,0,3.34,"13, SLV 320",O[C@H]1CC[C@H](Nc2nc(-c3ccccc3)nc3[nH]ccc23)CC1,Activity,'=',...,A549,Homo sapiens,CELL-LINE,CHEMBL4184095,1,Scientific Literature,Bioorg Med Chem Lett,2018.0,CHEMBL3307651,NaN
2,CHEMBL136689,NaN,None,389.42,0,2.96,"15, SCH 442416",COc1ccc(CCCn2ncc3c2nc(N)n2nc(-c4ccco4)nc32)cc1,Activity,'=',...,A549,Homo sapiens,CELL-LINE,CHEMBL4184095,1,Scientific Literature,Bioorg Med Chem Lett,2018.0,CHEMBL3307651,NaN
3,CHEMBL483688,NaN,None,529.02,1,2.65,"16, PSB 603",CCCn1c(=O)[nH]c2nc(-c3ccc(S(=O)(=O)N4CCN(c5ccc...,Activity,'=',...,A549,Homo sapiens,CELL-LINE,CHEMBL4184095,1,Scientific Literature,Bioorg Med Chem Lett,2018.0,CHEMBL3307651,NaN
4,CHEMBL4218062,NaN,None,523.59,1,2.81,30,CN(C)CCCNc1cc2nc(/C=C3\C(=O)Nc4cc(N5CCN(C)CC5)...,IC50,'=',...,A549,Homo sapiens,CELL-LINE,CHEMBL4177752,1,Scientific Literature,J Med Chem,2018.0,CHEMBL3307651,NaN


In [307]:
df.columns

Index(['Molecule ChEMBL ID', 'Smiles', 'Standard Type', 'Standard Relation',
       'Standard Value', 'Standard Units', 'pChEMBL Value',
       'Data Validity Comment', 'Assay Type', 'Assay Description',
       'Assay Organism', 'Assay Variant Mutation', 'Target Name',
       'Target Organism', 'Document Journal'],
      dtype='object')

# 2. FLOW

In [308]:
columns = ['Molecule ChEMBL ID', 'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value', 'Standard Units', 'pChEMBL Value', 'Data Validity Comment',
          'Assay Type', 'Assay Description','Assay Organism', 'Assay Variant Mutation','Target Name','Target Organism', 'Document Journal']

df = data[columns]

## 2.1 Target filter

In [309]:
df['Target Name'].unique()

array(['A549'], dtype=object)

In [310]:
df['Target Organism'].unique()

array(['Homo sapiens'], dtype=object)

In [311]:
def target_filter(data, name_col, target_name, org_col, target_org):
    df= data[data[name_col]==target_name]
    df = df[df[org_col]==target_org]
    return df
df1 = target_filter(data = df, name_col = 'Target Name', target_name ='A549', org_col='Target Organism', target_org = 'Homo sapiens')

## 2.2. Target value standardize (Standard)

In [312]:
class Target_standardize:
    def __init__(self, data, type_col, unit_col, active_col, relate_col, type_arg,equal_only = False, thresh = 7):
        self.data = data.reset_index(drop=True)
        self.type_col = type_col
        self.unit_col = unit_col
        self.active_col = active_col
        self.relate_col = relate_col
        self.type_arg = type_arg
        self.equal_only = equal_only
        self.thresh = thresh
      
    def standardize_value(self, data, type_col, type_arg, unit_col):
        df = data[data[type_col]==type_arg]
        df= df.dropna(subset =unit_col)
        df.reset_index(drop=True, inplace = True)
        type = ['μM','µM', 'nM', 'mM', 'M', 'nmol/l']
        idx = []
        for key, value in enumerate(df[unit_col]):
            if value in type:
                idx.append(key)
        df = df.iloc[idx,:]
        return df
    
    def convert_activity(self, data, active_col, unit_col):
        df = data.copy()
        df['pChEMBL'] = np.zeros(len(df))
        #unit = df['Unit'].unique()

        for key, value in enumerate(df[unit_col]):
            if value == 'μM':
                df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1e-6)
            elif value  == 'µM':
                df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1e-6)
            elif value  == 'nM':
                df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1e-9)
            elif value  == 'nmol/l':
                df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1e-9)
            elif value  == 'mM':
                df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1e-3)
            elif value  == 'M':
                df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1)
            elif value  == 'no unit':
                df.loc[key, 'pChEMBL'] = -df.loc[key, active_col]
        return df
    
    
    def standardize_relation(self, data,relate_col,  equal_only, thresh):
        df = data.copy()
        df.dropna(subset = relate_col, inplace = True)
        if equal_only == True:
            print('SELECTING ONLY EQUAL')
            df = df[df[relate_col]=="'='"]
            
        else:
            print('HANDLING')
            df_big = df[(df[relate_col] == "'>'") | (df[relate_col] == "'>='")]
            df_small = df[(df[relate_col] == "'<'") | (df[relate_col] == "'<='")]
            df_equal = df[df[relate_col]=="'='"]
                
            #Drop pCHEMBL < thresh for df_big
            drop_idx = df_big[df_big["pChEMBL"] < thresh].index
            df_big.drop(drop_idx, inplace = True)
                
            #Drop pCHEMBL > thresh for df_small
            drop_idx = df_small[df_small["pChEMBL"] > thresh].index
            df_small.drop(drop_idx, inplace = True)
                
            df = pd.concat((df_equal, df_small, df_big), axis = 0)
        return df
                  
    def fit(self):
        print("Number of data before standardizing:", self.data.shape[0])
        df = self.standardize_value(data=self.data, type_col=self.type_col, type_arg=self.type_arg, unit_col=self.unit_col)
        df.reset_index(drop=True, inplace = True)
        print("Number of data after select unit:", df.shape[0])
        #display(df.head(5))
        df2 = self.convert_activity(data=df, active_col=self.active_col, unit_col = self.unit_col)
        df2.reset_index(drop=True, inplace = True)
        #display(df2.head(5))
        df3 = self.standardize_relation(data=df2,relate_col=self.relate_col,  equal_only=self.equal_only, thresh=self.thresh)
        self.df = df3
        print("Number of data after standardizing:", self.df.shape[0])
        
        

In [313]:
target = Target_standardize(data =df1, type_col='Standard Type', unit_col='Standard Units', active_col='Standard Value', 
                         relate_col='Standard Relation', type_arg ='IC50',equal_only = False, thresh = 7)
target.fit()

Number of data before standardizing: 122196
Number of data after select unit: 39634


/tmp/ipykernel_675207/1065433019.py:35: RuntimeWarning: divide by zero encountered in log10
  df.loc[key, 'pChEMBL'] = -np.log10(df.loc[key, active_col]*1e-9)


HANDLING
Number of data after standardizing: 30581


/tmp/ipykernel_675207/1065433019.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_big.drop(drop_idx, inplace = True)
/tmp/ipykernel_675207/1065433019.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_small.drop(drop_idx, inplace = True)


In [314]:
df2 = target.df

## 2.3. Assay

In [315]:
class assay_curate:
    def __init__(self,data, type_col, org_col,des_col, type_arg='F', org_arg='Homo sapiens', kw = 'MTT'):
        self.data = data
        self.type_col = type_col
        self.org_col = org_col
        self.des_col = des_col
        self.type_arg= type_arg
        self.org_arg= org_arg
        self.kw = kw 
    
    def search_kw(self, data,kw, des_col):
        index = []
        for key, value in enumerate(data[des_col]):
            if kw in value:
                index.append(key)
        return data.iloc[index,:]
    
    def curate(self):
        print("Number of data befor standardizing:", self.data.shape[0])
        df = self.data[self.data[self.type_col]==self.type_arg]
        print("Number of data after choosing assay type:", df.shape[0])
        df2 = df[df[self.org_col]==self.org_arg]
        print("Number of data after choosing assay organism:", df2.shape[0])
        df3 = self.search_kw(data=df2, kw = self.kw, des_col = self.des_col)
        print("Number of data after curating:", df3.shape[0])
        self.df = df3

In [316]:
assay = assay_curate(data=df2, type_col="Assay Type", org_col="Assay Organism",des_col='Assay Description', 
                     type_arg='F', org_arg='Homo sapiens', kw = 'MTT')

assay.curate()

Number of data befor standardizing: 30581
Number of data after choosing assay type: 30008
Number of data after choosing assay organism: 30004
Number of data after curating: 17948


In [317]:
df3 = assay.df
df3.shape

(17948, 16)

## 2.4 Smiles curated

In [318]:
class smile_curate:
    def __init__(self, data, smile_col, pchem_col, keep = 'best'):
        self.data = data
        self.smile_col = smile_col
        self.pchem_col = pchem_col
        self.keep = keep
    def smile_norm(data, smile_col):
        df = data.dropna(subset = smile_col)
        df['Canonical_Smiles'] = df[smile_col].apply(Chem.CanonSmiles)
        return df
    def curate(self):
        df = smile_norm(data=self.data, smile_col=self.smile_col)
        if self.keep == 'best':
            df = df.sort_values(by=self.pchem_col, ascending=False)
            df_dropdup = df.drop_duplicates(subset=['Canonical_Smiles'], keep="first")
        elif self.keep == 'worst':
            df = df.sort_values(by=self.pchem_col, ascending=True)
            df_dropdup = df.drop_duplicates(subset=['Canonical_Smiles'], keep="first")
           
        print(df_dropdup.shape)
        self.df = df_dropdup

In [319]:
smile = smile_curate(data=df3, smile_col='Smiles', pchem_col='pChEMBL', keep = 'best')
smile.curate()

(16292, 17)


In [320]:
df4 = smile.df

In [321]:
df4.head()

,Molecule ChEMBL ID,Smiles,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Data Validity Comment,Assay Type,Assay Description,Assay Organism,Assay Variant Mutation,Target Name,Target Organism,Document Journal,pChEMBL,Canonical_Smiles
5001,CHEMBL314934,COc1ccc(-c2cc3c(OC)cc(OC)cc3nc2O)cc1,IC50,'=',0.000,nM,NaN,Outside typical range,F,Cytotoxicity against A549 nonsmall lung cell l...,Homo sapiens,NaN,A549,Homo sapiens,J Med Chem,inf,COc1ccc(-c2cc3c(OC)cc(OC)cc3nc2O)cc1
26804,CHEMBL455213,CC[C@H](C)[C@H](NC(=O)[C@H]1CCCCN1C)C(=O)N(COC...,IC50,'=',0.013,nM,10.89,NaN,F,Cytotoxicity against human A549 cells after 72...,Homo sapiens,NaN,A549,Homo sapiens,Bioorg Med Chem Lett,10.886057,CC[C@H](C)[C@H](NC(=O)[C@H]1CCCCN1C)C(=O)N(COC...
13070,CHEMBL2324626,COC(=O)c1cc(NC(=O)c2cc(-c3ccc(NC(=O)CCCOc4cc5c...,IC50,'=',0.019,nM,10.72,NaN,F,Cytotoxicity against human A549 cells after 96...,Homo sapiens,NaN,A549,Homo sapiens,J Med Chem,10.721246,COC(=O)c1cc(NC(=O)c2cc(-c3ccc(NC(=O)CCCOc4cc5c...
13459,CHEMBL4528553,C[C@@H](O)[C@@]12O[C@]13c1cc(O)c4c(c1N[C@H]2C#...,IC50,'=',0.020,nM,10.70,NaN,F,Cytotoxicity against human A549 cells assessed...,Homo sapiens,NaN,A549,Homo sapiens,J Nat Prod,10.698970,C[C@@H](O)[C@@]12O[C@]13c1cc(O)c4c(c1N[C@H]2C#...
13068,CHEMBL2324630,COC(=O)c1cc(-c2ccc(NC(=O)c3nc(NC(=O)CCCOc4cc5c...,IC50,'=',0.034,nM,10.47,NaN,F,Cytotoxicity against human A549 cells after 96...,Homo sapiens,NaN,A549,Homo sapiens,J Med Chem,10.468521,COC(=O)c1cc(-c2ccc(NC(=O)c3nc(NC(=O)CCCOc4cc5c...


In [322]:
df4.to_csv('A549.csv')